In [1]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [2]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [3]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [4]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [5]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [6]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2015-07-01 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


4712
169


In [7]:
print(training_set)

      problem_id  user_id status       submissionDate
0             10        5     AC  2014-02-17 15:27:07
1              2        6     AC  2014-02-17 15:39:17
2              2        9     AC  2014-02-18 00:30:14
3             10        9     AC  2014-02-18 00:34:46
4              4        9     AC  2014-02-18 00:50:28
5              6        9     AC  2014-02-18 00:52:11
6             13        9     AC  2014-02-18 00:53:40
9             15        8     AC  2014-02-19 19:58:03
10             4        8     AC  2014-02-20 14:23:30
11            39       16     AC  2014-02-20 15:44:33
12            39       17     AC  2014-02-20 16:53:34
13            13       12     AC  2014-02-21 11:08:38
15            33       12     AC  2014-02-21 11:58:32
16            39       12     AC  2014-02-21 12:05:49
17            44       12     AC  2014-02-21 12:52:44
18            44       15     AC  2014-02-21 14:51:52
19            44       22     AC  2014-02-21 14:58:55
21            19        8   

In [8]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2015-07-01 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
6933          117      725     WA  2015-07-01 09:17:38
6934          122      725    RTE  2015-07-01 11:09:50
6935            2     1665     CE  2015-07-01 13:15:06
6936          340     1504     WA  2015-07-01 16:12:24
6937          443     1735     TL  2015-07-01 18:55:30
6938           39     1737     AC  2015-07-01 20:36:18
6939          134     1317     AC  2015-07-02 13:37:18
6940          150     1504     AC  2015-07-02 19:43:11
6941          183     1504     AC  2015-07-02 20:37:12
6942           39     1736     AC  2015-07-03 00:00:53
6943          195     1504     AC  2015-07-03 03:53:16
6944          508     1504     AC  2015-07-03 04:19:22
6945            2     1504     AC  2015-07-03 04:29:37
6946          174     1504     AC  2015-07-03 17:16:06
6947           35      103     AC  2015-07-03 18:10:52
6948          181     1504     AC  2015-07-03 22:08:51
6949            8     1736     AC  2015-07-04 15:46:08
6950      

In [9]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

4443


In [10]:
def apply_aa(row, graph):
    """
        Funcion que devuelve para cada par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """
    
    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(row['one'], row['two'])])
    
    value_aa = 0
    for u, v, p in value:
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p
    
    return value_aa

def create_aa_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de aa para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_aa = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_aa['aa'] = dataFrame_aa.apply (lambda row: apply_aa(row, graph), axis=1)


    return dataFrame_aa

In [11]:
aa_df = create_aa_data(graph, nodes)
print(aa_df)

       one  two         aa
0       10    2  11.304202
1       10    4   9.811833
2       10    6   8.521302
3       10   13   9.431564
4       10   15  10.654518
5       10   39  11.311524
6       10   33   9.944589
7       10   44  10.162211
8       10   19   6.680407
9       10   27   5.449414
10      10   60   5.103790
11      10  100  10.519732
12      10   93   0.000000
13      10   53   9.234789
14      10   51   9.199781
15      10   49  10.850358
16      10   70   9.919868
17      10   62  10.955312
18      10   35   8.038641
19      10   81   1.066350
20      10    8   7.491605
21      10   17   1.747898
22      10   23   5.102796
23      10   25   0.000000
24      10   86   3.308712
25      10   29  10.167451
26      10   31   0.000000
27      10   47   0.000000
28      10   65   0.000000
29      10   73   6.412311
...    ...  ...        ...
14166  510  509   6.895348
14167  510  507   0.000000
14168  511  504   0.000000
14169  511  503   7.266353
14170  511  505   2.631301
1

In [12]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                       list_problem_id  user_id
0                                        [10, 76, 489]        5
1                                                  [2]        6
2                     [39, 44, 76, 105, 109, 114, 117]        7
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
5                                            [33, 109]       11
6    [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
7                                            [44, 187]       15
8                                                 [39]       16
9                                             [39, 86]       17
10                                                [44]       22
11                                           [100, 13]       24
12                 [258, 141, 309, 310, 390, 187, 510]       25
13                                               [100]       28
14                                      

In [13]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [14]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             3               2
1           6             1               1
2           9             9               5
3           8            40               8
4          16             1               0
5          17             2               2
6          12            12               0
7          18             0              13
8          15             2               2
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            40               2
16         33             7               2
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [15]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
2           9             9               5
3           8            40               8
29         60             6              27
118        25             7              33
261       130            62              34
274       382             5               6
285       206           111              38
286        62           112              32
299       414             9             103
310       437           134              13
316       443            41              22
317       448            31              33
333       381            13              10
348       103           109              48
383       542             8              13
395       431            11               8
483       689             5              41
597       254            11              12
601       864            14               8
619       912             7              17
621       840            26              19
633       935            88     

In [16]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                       list_problem_id  user_id
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
12                 [258, 141, 309, 310, 390, 187, 510]       25
37                            [100, 62, 2, 57, 53, 23]       60
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381
225                          [119, 262, 264, 114, 388]      382
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      431
248  [250, 124, 249, 100, 4, 122, 57, 30

In [17]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [18]:
def getSimilarProblems(row, aa_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor AA
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = aa_df[aa_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['aa'] > 0]
    column_result_two_tmp = aa_df[aa_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['aa'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_aa = list_one + list_two
    
    # print(list_aa)
    
    if list_aa == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_aa = graph.nodes()
        list_aa.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_aa

In [19]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, aa_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...
13           93  [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10

In [20]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17                              [35, 133, 129, 437, 103]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                             [35, 103]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [21]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_problems
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [22]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
dataframe_problem_recomend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [23]:
# elimino la columna de los problemas vecinos, ya que no interesa
del dataframe_problem_recomend['neighbors']

print(dataframe_problem_recomend)

     problem_id                                         list_users
0            10  [6, 33, 67, 56, 84, 60, 112, 124, 106, 41, 158...
1             2  [126, 39, 313, 42, 62, 46, 89, 152, 221, 88, 1...
2             4  [46, 67, 89, 137, 152, 231, 221, 88, 194, 205,...
3             6  [12, 59, 109, 54, 24, 55, 106, 136, 82, 42, 13...
4            13  [167, 205, 231, 224, 41, 452, 437, 448, 665, 8...
5            15  [16, 17, 12, 7, 40, 33, 66, 68, 71, 69, 70, 65...
6            39  [59, 152, 196, 271, 274, 269, 270, 11, 272, 27...
7            33  [15, 22, 7, 33, 129, 117, 202, 156, 54, 153, 2...
8            44  [8, 44, 205, 437, 527, 448, 1063, 1542, 1504, ...
9            19  [67, 167, 39, 440, 130, 488, 1261, 1130, 1603,...
10           27  [53, 84, 114, 55, 232, 133, 47, 191, 431, 1441...
11           60  [28, 29, 12, 32, 33, 49, 42, 46, 24, 60, 59, 5...
12          100  [8, 67, 31, 189, 383, 440, 137, 609, 377, 1441...
13           93  [79, 42, 247, 206, 84, 238, 409, 62, 249, 191

In [24]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [25]:

# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
dataframe_problem_recomend['lista_users_unique'] = dataframe_problem_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_problem_recomend)

     problem_id                                         list_users  \
0            10  [6, 33, 67, 56, 84, 60, 112, 124, 106, 41, 158...   
1             2  [126, 39, 313, 42, 62, 46, 89, 152, 221, 88, 1...   
2             4  [46, 67, 89, 137, 152, 231, 221, 88, 194, 205,...   
3             6  [12, 59, 109, 54, 24, 55, 106, 136, 82, 42, 13...   
4            13  [167, 205, 231, 224, 41, 452, 437, 448, 665, 8...   
5            15  [16, 17, 12, 7, 40, 33, 66, 68, 71, 69, 70, 65...   
6            39  [59, 152, 196, 271, 274, 269, 270, 11, 272, 27...   
7            33  [15, 22, 7, 33, 129, 117, 202, 156, 54, 153, 2...   
8            44  [8, 44, 205, 437, 527, 448, 1063, 1542, 1504, ...   
9            19  [67, 167, 39, 440, 130, 488, 1261, 1130, 1603,...   
10           27  [53, 84, 114, 55, 232, 133, 47, 191, 431, 1441...   
11           60  [28, 29, 12, 32, 33, 49, 42, 46, 24, 60, 59, 5...   
12          100  [8, 67, 31, 189, 383, 440, 137, 609, 377, 1441...   
13           93  [79

In [26]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation = dataframe_problem_recomend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['problem_id', 'possible_user']

print(df_separation)

        problem_id  possible_user
0                2            126
1                2             39
2                2            313
3                2             42
4                2             62
5                2             46
6                2             89
7                2            152
8                2            221
9                2             88
10               2            194
11               2             90
12               2             99
13               2            155
14               2            542
15               2            896
16               2           1186
17               2            598
18               2            584
19               2            596
20               2            594
21               2            593
22               2            581
23               2            585
24               2            597
25               2            582
26               2            589
27               2            600
28            

In [27]:
def getScore(row, dataframe_problem_recomend):
    """
        Funcion que devuelve el numero de apariciones del posible usuario en la lista de usuarios
    """
    # saco la lista de usuarios original, de donde voy a sacar la cuenta
    df_lista_usuarios_original = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    lista_usuarios_original = list(df_lista_usuarios_original['list_users'])[0]
    
    # print(lista_problemas_original)
    
    # saco el problema a contar
    user = row['possible_user']
    
    #print(problem)
    
    return lista_usuarios_original.count(user)

In [28]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

df_separation['score'] = df_separation.apply (lambda row: getScore(row, dataframe_problem_recomend), axis=1)

print(df_separation)

        problem_id  possible_user  score
0                2            126      8
1                2             39     10
2                2            313      1
3                2             42     23
4                2             62    109
5                2             46      8
6                2             89      1
7                2            152     15
8                2            221      3
9                2             88      2
10               2            194      1
11               2             90      4
12               2             99      3
13               2            155      6
14               2            542      8
15               2            896      9
16               2           1186      1
17               2            598      9
18               2            584     13
19               2            596     21
20               2            594      4
21               2            593      7
22               2            581      8
23              

In [29]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(df_separation)

        problem_id  possible_user  score
401              2           1733      1
1235             4           1733      1
2083             6           1733      1
3380             8           1733      1
3912            10           1733      1
4684            13           1733      1
5535            15           1733      1
6869            17           1733      1
6970            19           1733      1
8650            23           1733      1
9296            25           1733      1
9653            27           1733      1
11324           29           1733      1
11970           31           1733      1
12388           33           1733      1
13910           35           1733      1
14055           39           1733      1
14583           44           1733      1
15931           47           1733      1
17036           49           1733      1
17931           53           1733      1
19654           57           1733      1
19745           60           1733      1
21401           

In [30]:
print(df_separation[df_separation['score'] > 1])

        problem_id  possible_user  score
309              2           1722      4
1134             4           1722      4
1981             6           1722      4
2888             8           1722      4
3822            10           1722      4
4576            13           1722      4
5417            15           1722      4
6372            17           1722      4
7350            19           1722      4
8153            23           1722      4
9241            25           1722      4
10037           27           1722      4
10821           29           1722      4
11915           31           1722      4
12326           33           1722      4
13428           35           1722      4
15876           47           1722      4
16589           49           1722      4
17454           51           1722      4
18339           53           1722      4
19134           57           1722      4
20132           60           1722      4
20933           62           1722      4
21983           

In [31]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                            problem_id  user_id
0    [25, 31, 47, 65, 93, 166, 231, 239, 266, 269, ...        5
1    [4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...        6
2    [25, 31, 47, 65, 93, 166, 231, 239, 266, 269, ...        7
3    [25, 31, 47, 65, 93, 166, 231, 239, 266, 269, ...        8
4    [8, 15, 17, 19, 23, 25, 27, 29, 31, 33, 35, 39...        9
5    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       11
6    [2, 4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 3...       12
7    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       15
8    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       16
9    [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       17
10   [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       22
11   [2, 4, 6, 8, 10, 15, 17, 19, 23, 25, 27, 29, 3...       24
12   [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       25
13   [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...       28
14   [2, 4, 6, 8, 10, 13, 15, 17, 19, 23

In [32]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [33]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                       list_problem_id  user_id  \
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8   
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9   
12                 [258, 141, 309, 310, 390, 187, 510]       25   
37                            [100, 62, 2, 57, 53, 23]       60   
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62   
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103   
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105   
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206   
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254   
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381   
225                          [119, 262, 264, 114, 388]      382   
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414   
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      43

In [34]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

     user_id                                     recommendation
3          8  [25, 31, 47, 65, 93, 166, 231, 239, 266, 269, ...
4          9  [8, 15, 17, 19, 23, 25, 27, 29, 31, 33, 35, 39...
12        25  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...
37        60  [4, 6, 8, 10, 13, 15, 17, 19, 25, 27, 29, 31, ...
38        62  [25, 31, 47, 65, 76, 93, 231, 239, 269, 275, 3...
69       103  [31, 76, 93, 166, 231, 239, 266, 269, 275, 331...
71       105  [2, 4, 6, 8, 13, 15, 17, 19, 23, 25, 27, 29, 3...
88       130  [25, 31, 47, 65, 76, 93, 166, 266, 269, 275, 3...
135      206  [25, 31, 47, 65, 76, 93, 239, 269, 275, 331, 3...
162      254  [2, 4, 6, 8, 10, 13, 15, 19, 23, 25, 27, 29, 3...
224      381  [25, 31, 39, 47, 65, 70, 76, 93, 105, 114, 117...
225      382  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...
237      414  [4, 6, 8, 13, 15, 17, 19, 23, 25, 27, 29, 31, ...
245      431  [2, 4, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ...
248      437  [25, 47, 484, 486, 489, 50

In [35]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [36]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

     user_id                                     recommendation  \
3          8  [25, 31, 47, 65, 93, 166, 231, 239, 266, 269, ...   
4          9  [8, 15, 17, 19, 23, 25, 27, 29, 31, 33, 35, 39...   
12        25  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...   
37        60  [4, 6, 8, 10, 13, 15, 17, 19, 25, 27, 29, 31, ...   
38        62  [25, 31, 47, 65, 76, 93, 231, 239, 269, 275, 3...   
69       103  [31, 76, 93, 166, 231, 239, 266, 269, 275, 331...   
71       105  [2, 4, 6, 8, 13, 15, 17, 19, 23, 25, 27, 29, 3...   
88       130  [25, 31, 47, 65, 76, 93, 166, 266, 269, 275, 3...   
135      206  [25, 31, 47, 65, 76, 93, 239, 269, 275, 331, 3...   
162      254  [2, 4, 6, 8, 10, 13, 15, 19, 23, 25, 27, 29, 3...   
224      381  [25, 31, 39, 47, 65, 70, 76, 93, 105, 114, 117...   
225      382  [2, 4, 6, 8, 10, 13, 15, 17, 19, 23, 25, 27, 2...   
237      414  [4, 6, 8, 13, 15, 17, 19, 23, 25, 27, 29, 31, ...   
245      431  [2, 4, 8, 10, 13, 15, 17, 19, 23, 25, 27, 29, ..

In [37]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

     user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
3          8               [25]           [25, 31]       [25, 31, 47]   
4          9                [8]            [8, 15]        [8, 15, 17]   
12        25                [2]             [2, 4]          [2, 4, 6]   
37        60                [4]             [4, 6]          [4, 6, 8]   
38        62               [25]           [25, 31]       [25, 31, 47]   
69       103               [31]           [31, 76]       [31, 76, 93]   
71       105                [2]             [2, 4]          [2, 4, 6]   
88       130               [25]           [25, 31]       [25, 31, 47]   
135      206               [25]           [25, 31]       [25, 31, 47]   
162      254                [2]             [2, 4]          [2, 4, 6]   
224      381               [25]           [25, 31]       [25, 31, 39]   
225      382                [2]             [2, 4]          [2, 4, 6]   
237      414                [4]             [4, 6] 

In [38]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
2              [445, 436, 437, 441, 442, 443, 444, 62]        8
3                             [83, 109, 136, 141, 254]        9
9    [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...       25
16   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...       60
17   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...       62
21   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...      103
22   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...      105
24   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...      130
30   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...      206
34   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...      254
43    [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]      381
44                              [33, 8, 465, 6, 2, 13]      382
46   [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...      414
50             [253, 33, 252, 443, 444, 441, 404, 503]      431
51   [489, 181, 390, 393, 397, 437, 486,

In [39]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0             [445, 436, 437, 441, 442, 443, 444, 62]             [25]   
1                            [83, 109, 136, 141, 254]              [8]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...              [2]   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...              [4]   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...             [25]   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...             [31]   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...              [2]   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...             [25]   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...             [25]   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...              [2]   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]             [25]   
11                             [33, 8, 465, 6, 2, 13]              [2]   
12  [239, 62, 187, 206, 313, 316, 504,

In [40]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [41]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [42]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [43]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [44]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [45]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0             [445, 436, 437, 441, 442, 443, 444, 62]             [25]   
1                            [83, 109, 136, 141, 254]              [8]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...              [2]   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...              [4]   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...             [25]   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...             [31]   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...              [2]   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...             [25]   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...             [25]   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...              [2]   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]             [25]   
11                             [33, 8, 465, 6, 2, 13]              [2]   
12  [239, 62, 187, 206, 313, 316, 504,

In [46]:


f = open("C:/hlocal/TFM/vot_simple", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.21428571428571427
Precision ----------
0.21428571428571427
Mrr  ----------
0.21428571428571427
Recall  ----------
0.012566288079090287
F1  ----------
0.02309958171302709
2
###########
One hit ----------
0.2619047619047619
Precision ----------
0.15476190476190477
Mrr  ----------
0.23809523809523808
Recall  ----------
0.015205457162434129
F1  ----------
0.026379800449025232
3
###########
One hit ----------
0.3333333333333333
Precision ----------
0.1587301587301587
Mrr  ----------
0.2619047619047619
Recall  ----------
0.028535126221992228
F1  ----------
0.04449744385010347
4
###########
One hit ----------
0.40476190476190477
Precision ----------
0.16071428571428573
Mrr  ----------
0.27976190476190477
Recall  ----------
0.037154188949703404
F1  ----------
0.054341293414119404
5
###########
One hit ----------
0.42857142857142855
Precision ----------
0.1666666666666667
Mrr  ----------
0.28452380952380957
Recall  ----------
0.044987309224805455
F1  ---------